In [1]:
!pip install openpyxl

In [2]:
import os
import requests
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
from selenium import webdriver
import os
from tqdm import tqdm
from random import random
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
from undetected_chromedriver import Chrome, ChromeOptions
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException

In [3]:
url = 'https://pe.computrabajo.com'

In [4]:
works = {'DS':'Data science',
         'DA':'Data analyst',
         'DE':'Data engineer',
         'BI':'Business Intelligence'}

In [5]:
def web_scrap(url,b, lugares):
    if b=='dptos':
        options = ChromeOptions()
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        driver = Chrome(options=options)
        driver.get(url)
        lgr = []
        lgr_2 = []
        text_emp = 'Empleos en '
        while True:
            # Esperar hasta que aparezca al menos un elemento 'list'
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, "cI")))   
            # Encontrar todos los elementos 'list'
            lists = driver.find_elements(By.CLASS_NAME, "cI")
            # Recorrer cada 'list' y extraer la información requerida
            for list_element in lists:
                lugares = list_element.find_element(By.TAG_NAME, "a").text
                if len(lugares)==0:
                    break
                lugares = lugares[len(text_emp)::]
                lgr.append(lugares)
                a = ''
                if ' ' in lugares: 
                    a = lugares.split(' ')
                    lugares = '-'.join(a)
                lgr_2.append(lugares.lower())
            break
        # Cerrar el navegador
        driver.quit()
        return lgr, lgr_2
    
    elif b=='empleos':
        for i in range(len(works)): 
            if ' ' in list(works.values())[i]: 
                a = works[list(works.keys())[i]].split(' ')
                works[list(works.keys())[i]] = '-'.join(a)
            works[list(works.keys())[i]] = works[list(works.keys())[i]].lower()
            
        wp = '/trabajo-de-'
        # Listas para almacenar la información
        info = {}
        for i in range(len(lugares)):
            genl = []
            for j in range(len(works)):
                options = ChromeOptions()
                options.add_argument("--no-sandbox")
                options.add_argument("--disable-dev-shm-usage")
                driver = Chrome(options=options)
                driver.get(url+wp+works[list(works.keys())[j]]+'-en-'+lugares[i])
            
                job = []
                companies = []
                urgency = []
                destacado = []
                modalidad = []
                dates = []

                while True:
                    # Esperar hasta que aparezca al menos un elemento 'list'
                    try:
                        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CLASS_NAME, "box_offer")))
                    except TimeoutException:
                        break
                    # Encontrar todos los elementos 'list'
                    lists = driver.find_elements(By.CLASS_NAME, "box_offer")
                    # Recorrer cada 'list' y extraer la información requerida
                    for list_element in lists:
                        jobs = list_element.find_element(By.CLASS_NAME, "js-o-link").text
                        company = list_element.find_element(By.CSS_SELECTOR, 'p.fc_base').text
                        try: 
                            urgencia = list_element.find_element(By.CSS_SELECTOR, "span.fc_urgent").text
                        except NoSuchElementException:
                            urgencia = ''
                        try: 
                            modo = list_element.find_element(By.CSS_SELECTOR, "span.tag.base.mb10").text
                        except NoSuchElementException:
                            modo = ''
                        try: 
                            destacados = list_element.find_element(By.CLASS_NAME, "fc_dest").text
                        except NoSuchElementException:
                            destacados = ''
                        #location = list_element.find_element(By.CLASS_NAME, "companyLocation").text
                        date = list_element.find_element(By.CLASS_NAME, "fc_aux").text
    
                        # Agregar la información a las listas correspondientes
                        job.append(jobs)
                        companies.append(company)
                        modalidad.append(modo)
                        urgency.append(urgencia)
                        destacado.append(destacados)
                        #locations.append(location)
                        dates.append(date)
    
                    # Verificar si existe el botón de siguiente página
                    try:
                        next_button = driver.find_element(By.XPATH, '//span[@title="Siguiente"]')
                        if 'disabled' in next_button.get_attribute("class"):
                            break
                    except NoSuchElementException:
                        break
        
        
                    time.sleep(random.uniform(3, 6))
    
                    # Hacer clic en el botón de siguiente página
                    driver.execute_script("arguments[0].click();", next_button)
                    WebDriverWait(driver, 30).until(EC.staleness_of(lists[0]))

                # Cerrar el navegador
                driver.quit()
                
                # Crear un dataframe con la información recolectada
                data_ds = {
                    'Company': companies,
                    'Job': job,
                    'Modalidad': modalidad,
                    'Urgency': urgency,
                    'Destacado': destacado,
                    'Date': dates,
                    }

                df_ds = pd.DataFrame(data_ds)
                campos_revisión = ['Company', 'Job', 'Date']
                df_ds1 = df_ds.drop_duplicates(subset=campos_revisión).reset_index(drop=True)
                df_ds1.index = df_ds1.index + 1

                genl.append(df_ds1)
                
            info[i] = genl
        return info

In [6]:
departamentos = web_scrap(url, 'dptos', '')

In [7]:
empleos = web_scrap(url, 'empleos', departamentos[1])

In [16]:
regiones = {}
for i in range(len(departamentos[0])):
    regiones[departamentos[0][i]]=pd.concat(empleos[i],axis=0,keys=list(works.keys()))

In [18]:
fecha = datetime.today().strftime('%Y-%m-%d')

path = '/'.join(os.getcwd().split('\\'))

for i in range(len(departamentos[0])):
    regiones[departamentos[0][i]].to_excel('%s/%s-%s.xlsx'%(path,fecha,departamentos[0][i]), index=True)